In [1]:
import random
import re
import pandas as pd
import os

def lm_emulator(prompt):
    test_text = f"""
        Ich werde die bereitgestellten Daten analysieren und eine Einschätzung zur Wirkung der Impfkampagne auf die Infektionsrate in NRW im Januar 2022 vornehmen.

        1. Analyse der aktuellen Situation:
        Infektionsrate: 0.5
        Fallzahlen: 1000
        Demografische Impftrends: Frauen werden häufiger geimpft als Männer. Menschen mit Hypertonie haben eine höhere Impfquote, besonders über 60-Jährige.
        Regionale Impfquote: NRW hat die höchste Impfquote unter den Bundesländern.
        Bevölkerungsstruktur: NRW hat eine hohe Bevölkerungsdichte, was die Virusübertragung begünstigt.
        Impfquote nach Alter: Ältere Menschen, besonders über 60, werden häufiger geimpft, was den Schutz der vulnerablen Gruppen verbessert.
        2. Wissenschaftliche Einschätzung der Impfkampagnenwirkung:
        Impfkampagnen führen kurzfristig zu einer Reduktion der Infektionsrate, da mehr Menschen immunisiert werden, was die Übertragung reduziert.
        Herdenimmunitätseffekt: Da die Impfquote unter älteren Menschen hoch ist, wird erwartet, dass schwere Krankheitsverläufe reduziert werden.
        Geschlechtsspezifischer Einfluss: Männer haben eine niedrigere Impfquote, was potenziell zu einer höheren Infektionsrate in dieser Gruppe führen könnte.
        Einfluss der Privatversicherung: Da Männer tendenziell häufiger privat versichert sind, könnte eine unterschiedliche Impfpriorisierung eine Rolle spielen.
        3. Prognose für die Infektionsrate in NRW nach der Impfkampagne:
        Die Impfkampagne wird voraussichtlich die Infektionsrate senken, jedoch nicht sofort drastisch, da Impfungen einige Zeit benötigen, um eine Immunreaktion zu erzeugen. Basierend auf bisherigen Impfkampagnen und der aktuellen Ausgangslage, könnte die Infektionsrate um 10–20% gesenkt werden.

        Geschätzte Infektionsrate nach der Impfkampagne:
        Infektionsrate: [{round(random.uniform(0.1, 2), 2)}]
    """
    return test_text

In [2]:
from sqlalchemy import create_engine, Column, String, Float, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class CacheEntry(Base):
    __tablename__ = 'cache_entries'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    Region = Column(String)
    Year = Column(Integer)
    Month = Column(Integer)
    InfluenzaCasesPerCapita = Column(Float)
    InfectionRate = Column(Float)
    PreviousInfectionRate = Column(Float)
    Prompt = Column(String)
    Output = Column(String)

# Initialize the database
engine = create_engine('sqlite:///cache.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

class Cache:
    def __init__(self):
        """Initialize the cache with SQLAlchemy session."""
        self.session = Session()

    def add_to_cache(self, region, year, month, influenza_cases_per_capita, infection_rate, prompt, output, previous_infection_rate=None):
        """Add or update a row in the cache."""

            # Add new entry
        entry = CacheEntry(
            Region=region,
            Year=int(year),
            Month=int(month),
            InfluenzaCasesPerCapita=float(influenza_cases_per_capita),
            InfectionRate=float(infection_rate),
            PreviousInfectionRate=float(previous_infection_rate) if previous_infection_rate else None,
            Prompt=prompt,
            Output=output
        )
        self.session.add(entry)
        self.session.commit()

    def get_from_cache(self, region, year, month):
        """Retrieve a value from the cache."""
        entry = self.session.query(CacheEntry).filter_by(Region=region, Year=year, Month=month).first()
        if entry:
            return {
                'Region': entry.Region,
                'Year': entry.Year,
                'Month': entry.Month,
                'InfluenzaCasesPerCapita': entry.InfluenzaCasesPerCapita,
                'InfectionRate': entry.InfectionRate,
                'PreviousInfectionRate': entry.PreviousInfectionRate,
                'Prompt': entry.Prompt,
                'Output': entry.Output
            }
        return None


C:\Users\Amaan\AppData\Local\Temp\ipykernel_12768\2933687082.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
def extract_between_brackets(text):
    """
    Extracts and returns the content inside square brackets from a string.
    If multiple brackets exist, it returns a list of all matches.

    :param text: The input string.
    :return: A string if one match, a list if multiple matches, or None if no match.
    """
    matches = re.findall(r'\[(.*?)\]', text)  # Extract content inside square brackets
    if not matches:
        return None  # Return None if no matches found
    return matches if len(matches) > 1 else matches[0]  # Return list if multiple, string if one


In [4]:
def parse_lm_output(output):
    """
    Parses the output from the language model and extracts the predicted values.

    :param output: The output text from the language model.
    :return: The predicted value(s) as a string or list of strings.
    """

    rate_str = extract_between_brackets(output)
    if rate_str is not None:
        if type(rate_str) != list:
            try:
                return float(rate_str)
            except ValueError:
                return None
        else:
            try:
                return float(rate_str[0])
            except ValueError:
                return None
    return

In [5]:
def generate_prompt(bundesland, jahr, monat, fallzahlen, infektionsrate):
    general_prompt = f"Wir führen eine Impfkampagne durch, um die Infektionsrate in {bundesland} im Jahr {jahr} und Monat {monat} zu senken. Analysiere die bereitgestellten Daten zur aktuellen Impfquote, Infektionsrate und weiteren relevanten Faktoren. Berücksichtige dabei dein Wissen über die Region, einschließlich Demografie, Gesundheitsinfrastruktur und bisheriger Impfeffekte. Schätze ein, wie sich die Impfkampagne voraussichtlich auf die Infektionsrate auswirken wird, und begründe deine Einschätzung mit Daten und wissenschaftlichen Erkenntnissen. Abschließend, am Ende deiner Ausgabe, gebe deine geschätzte Infektionsrate für den Zeitraum und das Bundesland, umschlossen von eckigen Klammern zurück d.h. z.B. Infektionsrate: [0.3].\n"

    data_knowledge = f"Die Infektionszahlen in {bundesland} im Jahr {jahr} und Monat {monat}. Die Fallzahlen sind {round(fallzahlen, 2)}. Die Infektionsrate des letzen Monats beträgt {round(infektionsrate, 2)}.\n"

    exploration_knowledge = ("Wir haben bereits folgendes rausgefunden: \n"
                             "- Frauen werden deutlich häufiger geimpft als Männer.\n"
                             "- Tendenziell sind Männer leicht häufiger privat versichert als Frauen.\n"
                             "- Menschen, die an Hypertonie erkrankt sind, werden häufiger geimpft als Menschen mit anderen Vorerkrankungen.\n"
                             "- Die Impfquote ist in der Altersgruppe über 60 am höchsten.\n"
                             "- Innerhalb der Menschen mit Risikogruppe, werden über 60 jährige exponentiell häufiger geimpft als unter 60 jährige. Dies gilt nur linear für Patient mit Astma\n"
                             "- Wir wissen, unter Beachtung der Bevölkerungsdichte, dass die Impfquote in den verschiedenen Bundesländern sehr ähnlich ist.\n"
                             "- Unter Beachtung der Bevölkerungsdichte, nimmt die Impfquote linear von 7,8% auf 4,3% ab. Dabei sind die Bundesländer absteigend wie folgt gerankt: Nordrhein-Westfahlen, Niedersachsen, Hessen, Baden-Württemberg, Bayern, Bremen, Sachsen-Anhalt, Schleswig-Holstein, Rheinland-Pfalz, Thüringen, Hamburg, Brandenburg, Berlin, Mecklenburg-Vorpommern, Sachsen, Saarland.\n"
                             )

    return general_prompt + data_knowledge + exploration_knowledge


In [7]:
#generate_prompt("NRW", "2022", "Januar", "1000", "0.5")

In [8]:
df = pd.read_csv("merged.csv", index_col=0, header=0)
df

,specialization,Year,CalendarWeek,absolute,extrapolated,Region,InfluenzaCases,Population,InfluenzaCasesPerCapita,VaccinationsPerCapita
0,1,2024,31,1,44,Baden-Württemberg,16,11230740,0.000001,0.000004
1,1,2024,32,4,180,Baden-Württemberg,13,11230740,0.000001,0.000016
2,1,2024,33,3,135,Baden-Württemberg,17,11230740,0.000002,0.000012
3,1,2024,36,1,45,Baden-Württemberg,19,11230740,0.000002,0.000004
4,1,2024,37,66,2952,Baden-Württemberg,19,11230740,0.000002,0.000263
...,...,...,...,...,...,...,...,...,...,...
916,3,2025,1,23,1104,Thüringen,139,2114870,0.000066,0.000522
917,3,2025,2,55,2857,Thüringen,486,2114870,0.000230,0.001351
918,3,2025,3,41,2132,Thüringen,871,2114870,0.000412,0.001008
919,3,2025,4,26,1352,Thüringen,1893,2114870,0.000895,0.000639


In [9]:
df['Date'] = pd.to_datetime(df['Year'].astype(str) + df['CalendarWeek'].astype(str) + '1', format='%Y%W%w')
df['Month'] = df['Date'].dt.month
df = df.drop(columns=['Date'])

In [10]:
dff = df.groupby(['Year', 'Month', 'Region']).sum().reset_index()

In [11]:
cache = Cache()

def send_row_to_model(row):
    previous_record = cache.get_from_cache(row['Region'], row['Year'], row['Month'] - 1)
    previous_rate = previous_record['InfectionRate'] if previous_record else 0

    prompt = generate_prompt(row['Region'], row['Year'], row['Month'], row['InfluenzaCasesPerCapita'], previous_rate)
    
    model_output = lm_emulator(prompt)
    parsed_value = parse_lm_output(model_output)

    cache.add_to_cache(
        row['Region'], 
        row['Year'], 
        row['Month'], 
        row['InfluenzaCasesPerCapita'], 
        parsed_value,  # Assuming infection rate is the parsed value
        prompt, 
        model_output,
        previous_rate
    )
    
    return parsed_value

In [12]:
send_row_to_model(dff.iloc[0])

1.31

In [13]:
dff.iloc[0]

Year                                    2024
Month                                      7
Region                     Baden-Württemberg
specialization                             4
CalendarWeek                              62
absolute                                   2
extrapolated                             103
InfluenzaCases                            32
Population                          22461480
InfluenzaCasesPerCapita             0.000003
VaccinationsPerCapita               0.000009
Name: 0, dtype: object

In [14]:
cache.get_from_cache("Baden-Württemberg", 2024, 7)

{'Region': 'Baden-Württemberg',
 'Year': 2024,
 'Month': 7,
 'InfluenzaCasesPerCapita': 2.8493224845379733e-06,
 'InfectionRate': 1.31,
 'PreviousInfectionRate': None,
 'Prompt': 'Wir führen eine Impfkampagne durch, um die Infektionsrate in Baden-Württemberg im Jahr 2024 und Monat 7 zu senken. Analysiere die bereitgestellten Daten zur aktuellen Impfquote, Infektionsrate und weiteren relevanten Faktoren. Berücksichtige dabei dein Wissen über die Region, einschließlich Demografie, Gesundheitsinfrastruktur und bisheriger Impfeffekte. Schätze ein, wie sich die Impfkampagne voraussichtlich auf die Infektionsrate auswirken wird, und begründe deine Einschätzung mit Daten und wissenschaftlichen Erkenntnissen. Abschließend, am Ende deiner Ausgabe, gebe deine geschätzte Infektionsrate für den Zeitraum und das Bundesland, umschlossen von eckigen Klammern zurück d.h. z.B. Infektionsrate: [0.3].\nDie Infektionszahlen in Baden-Württemberg im Jahr 2024 und Monat 7. Die Fallzahlen sind 0.0. Die Infekt

In [15]:
dff['predicted_value'] = dff.apply(lambda x: send_row_to_model(x), axis=1)

In [16]:
dff = dff[dff['Region'] == 'Baden-Württemberg']
dff.sort_values(by=['Year', "Month"], inplace=True)

In [17]:
dff

,Year,Month,Region,specialization,CalendarWeek,absolute,extrapolated,InfluenzaCases,Population,InfluenzaCasesPerCapita,VaccinationsPerCapita,predicted_value
0,2024,7,Baden-Württemberg,4,62,2,103,32,22461480,0.000003,0.000009,0.30
2,2024,8,Baden-Württemberg,2,65,7,315,30,22461480,0.000003,0.000028,1.16
14,2024,9,Baden-Württemberg,28,534,2768,146355,302,157230360,0.000027,0.013032,0.70
30,2024,10,Baden-Württemberg,24,510,13650,716986,366,134768880,0.000033,0.063841,1.53
46,2024,11,Baden-Württemberg,24,558,8296,435901,522,134768880,0.000046,0.038813,1.17
62,2024,12,Baden-Württemberg,22,554,2397,129133,2222,123538140,0.000198,0.011498,1.37
78,2025,1,Baden-Württemberg,22,29,677,36528,18104,123538140,0.001612,0.003253,1.47
94,2025,2,Baden-Württemberg,6,15,135,7240,12669,33692220,0.001128,0.000645,0.16


In [18]:
dff['previous'] = dff.apply(lambda x: cache.get_from_cache(x['Region'], x['Year'], x['Month'])['PreviousInfectionRate'], axis=1)

In [19]:
dff

,Year,Month,Region,specialization,CalendarWeek,absolute,extrapolated,InfluenzaCases,Population,InfluenzaCasesPerCapita,VaccinationsPerCapita,predicted_value,previous
0,2024,7,Baden-Württemberg,4,62,2,103,32,22461480,0.000003,0.000009,0.30,NaN
2,2024,8,Baden-Württemberg,2,65,7,315,30,22461480,0.000003,0.000028,1.16,1.31
14,2024,9,Baden-Württemberg,28,534,2768,146355,302,157230360,0.000027,0.013032,0.70,1.16
30,2024,10,Baden-Württemberg,24,510,13650,716986,366,134768880,0.000033,0.063841,1.53,0.70
46,2024,11,Baden-Württemberg,24,558,8296,435901,522,134768880,0.000046,0.038813,1.17,1.53
62,2024,12,Baden-Württemberg,22,554,2397,129133,2222,123538140,0.000198,0.011498,1.37,1.17
78,2025,1,Baden-Württemberg,22,29,677,36528,18104,123538140,0.001612,0.003253,1.47,NaN
94,2025,2,Baden-Württemberg,6,15,135,7240,12669,33692220,0.001128,0.000645,0.16,1.47
